In [21]:
#!pip install -q keras==2.3.0
%tensorflow_version 2.x
try:
  import tensorflow.compat.v2 as tf
except Exception:
  pass

tf.enable_v2_behavior()

print(tf.__version__)

#import tensorflow as tf
import tensorflow_hub as hub
print(tf.version.VERSION)
print(tf.keras.__version__)

2.0.0
2.0.0
2.2.4-tf


In [22]:
#!apt-get install graphviz
#!pip install graphviz
!pip uninstall -y hiddenlayer
!pip install git+https://github.com/LanceNorskog/hiddenlayer.git
import hiddenlayer as hl
import hiddenlayer.transforms as ht
from google.colab import files

Uninstalling hiddenlayer-0.2:
  Successfully uninstalled hiddenlayer-0.2
  Cloning https://github.com/LanceNorskog/hiddenlayer.git to /tmp/pip-req-build-u3ala0lf
  Running command git clone -q https://github.com/LanceNorskog/hiddenlayer.git /tmp/pip-req-build-u3ala0lf
  Created wheel for hiddenlayer: filename=hiddenlayer-0.2-cp36-none-any.whl size=19831 sha256=f9893b6f51af622c0af8615619b7d1945e014adddb9843fbb889a66aa55f7eb8
  Stored in directory: /tmp/pip-ephem-wheel-cache-vbticf5t/wheels/07/b1/ab/561eb5cc54236e7e06f4071b36f1265fce591196717cec58f6
Successfully built hiddenlayer


In [0]:
import math
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

# Read sklearn image set and:
# 1) choose a subset of image pairs
# 2) generate boolean values for "same digit" and "digit less than digit"
def get_data(num_samples=100000):
  digits = load_digits()
  num_images = digits.data.shape[0]
    
  # create pairs of all images VS all images
  num_pairs = num_images * num_images
  row, col = np.indices((num_images, num_images))
  pair1, pair2 = row.reshape(num_pairs), col.reshape(num_pairs)
    
  # randomize image pairs and possibly choose a subset
  scramble = np.arange(num_pairs, dtype='int32')
  np.random.shuffle(scramble)
  pair1 = pair1[scramble]
  pair2 = pair2[scramble]
  if num_samples < num_pairs:
    pair1 = pair1[0:num_samples]
    pair2 = pair2[0:num_samples]
    num_pairs = num_samples

  # prepare actual images for input
  input1 = digits.data[pair1] / 16.0
  input2 = digits.data[pair2] / 16.0

  # Are images of same digit? Is one digit less than the other?
  # (2 images, [0, 1])
  left = digits.target[pair1]
  right = digits.target[pair2]   
  output = np.reshape(np.transpose([left == right, left < right], (1, 0)), (num_pairs, 2))
    
  return input1, input2, output, left, right

X1_train,X2_train,Y_train, left, right = get_data(num_samples=100000)
X1_val = X1_train[90000:]
X2_val = X2_train[90000:]
Y_val = Y_train[90000:]
X1_train = X1_train[:90000]
X2_train = X2_train[:90000]
Y_train = Y_train[:90000]

In [0]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.python.keras.engine.base_layer import Layer
import scipy.stats as stats
import numpy as np

class NeuralTensorDiagLayer(Layer):
  def __init__(self, output_dim, activation=K.tanh, collector=K.mean, 
               feedforward=True, bias=True, ntn=True, siamese=True, **kwargs):
    self.output_dim = output_dim #k
    self.activation = activation
    self.collector = collector
    self.feedforward = feedforward
    self.bias = bias
    self.ntn = ntn
    self.siamese = siamese
    super(NeuralTensorDiagLayer, self).__init__(**kwargs)

  def build(self, input_shape):
    assert input_shape[0][-1] == input_shape[1][-1]
    mean = 0.0
    std = 1.0
    # W : k*d
    k = self.output_dim
    d = input_shape[0][-1]
    self.input_dim = d

    # w_init = tf.initializers.truncated_normal(mean=mean, stddev=2*std)
    # w_init =  tf.compat.v2.initializers.TruncatedNormal(mean=mean, stddev=2*std)
    #v_init = tf.initializers.truncated_normal(mean=mean, stddev=2*std)
    w_init = tf.initializers.glorot_uniform
    v_init = tf.initializers.glorot_uniform
    b_init = tf.initializers.zeros
    if self.ntn:
        self.W = self.add_weight(shape=(d, k), 
                             initializer=w_init,
                             trainable=True,
                             name='W')
        print('self.W:', self.W)
    if self.siamese:
        self.V = self.add_weight(shape=(d, k), 
                             initializer=v_init,
                             trainable=True,
                             name='V')
    elif self.feedforward:
        self.V = self.add_weight(shape=(2*d, k), 
                             initializer=v_init,
                             trainable=True,
                             name='V')
    if self.bias:
        self.b = self.add_weight(shape=(k), initializer=b_init, trainable=True, name='b')

  def call(self, inputs, mask=None):
    if type(inputs) is not list or len(inputs) != 2:
      raise Exception('NTNDiagLayer must be called on a list of tensors '
                      '(at least 2). Got: ' + str(inputs))
    k = self.output_dim
    e1 = inputs[0]
    e2 = inputs[1]
    if self.siamese:
        print('sub:', e1 - e2)
        feed_forward_product = K.dot(e1 - e2, self.V)
        print('ff: ', feed_forward_product)
    elif self.feedforward:
        feed_forward_product = K.dot(K.concatenate([e1,e2]), self.V)
        print('ff: ', feed_forward_product)
    if not self.ntn:
        result = feed_forward_product + self.b
        if self.activation:
            result = self.activation(result)
        return result

    e1 = K.flatten(inputs[0])
    e2 = K.flatten(inputs[1])
    e1 = K.tile(e1, k)
    e2 = K.tile(e2, k)
    print('e1:', e1)
    e1 = K.reshape(e1, shape=(-1, k, self.input_dim))
    e2 = K.reshape(e2, shape=(-1, k, self.input_dim))
    print('e1:', e1)
    # x0 = e1 * self.W
    # print('x0:', x0)
    # x1 = x0 * e2
    # print('x1:', x1)
    #x0 = K.dot(e2[...], self.W[...])
    #print('x0:', x0)
    #x = K.dot(e1, x0)
    #print('x:', x)
    x0 = K.dot(e1, self.W[...])
    print('x0:', x0)
    x = K.dot(e1, x0)
    print('x:', x)

    #y = self.collector(x, axis=-1, keepdims=True)
    #y = x
    #print('y:', y)
    #z = K.squeeze(y, axis=-1)
    #print('z:', z)
    stacked = x 
    print('o1: ', stacked)
    #stacked = K.expand_dims(stacked, axis=0)
    print('o2: ', stacked)
    if (self.feedforward or self.siamese) and self.bias:
        result = stacked + feed_forward_product + self.b
    elif (self.feedforward or self.siamese):
        result = stacked + feed_forward_product  
    elif self.bias:
        result = stacked + self.b
    elif not self.ntn:
        result = feedforward + self.b
    else:
        retult = None
    if self.activation:
        result = self.activation(result)
        
    print('result: ', result)
    print("call() finished")
    return result

  def compute_output_shape(self, input_shape):
    batch_size = input_shape[0][0]
    return (batch_size, self.output_dim)

  # not sure if valid when not base types
  def get_config(self):
    config = super(NeuralTensorDiagLayer, self).get_config()
    config.update({'output_dim': self.output_dim})
    config.update({'activation': self.activation})
    config.update({'collector': self.collector})
    config.update({'bias': self.bias})
    config.update({'feedforward': self.feedforward})
    return config

In [33]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

g = tf.Graph()
with g.as_default():
  activ = K.tanh
  collect = K.mean
  input1 = Input(shape=(64,), dtype='float32')
  input2 = Input(shape=(64,), dtype='float32')
  i1 = input1
  i2 = input2
  squishify = Dense(24) #, activation='tanh')
  i1 = squishify(input1)
  i2 = squishify(input2)
#   i1 = Dense(24, activation='tanh')(i1)
#   i2 = Dense(24, activation='tanh')(i2)
  btp = NeuralTensorDiagLayer(output_dim=32, 
                            activation=K.tanh, collector=K.mean, 
                            feedforward=False, siamese=False, ntn=True)([i1, i2])
  #last = Dense(units=16, activation='tanh')(btp)
  p = Dense(units=2, activation='sigmoid')(btp)
    
  model = Model(inputs=[input1, input2], outputs=[p])

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
  model.summary()

  early = EarlyStopping(monitor='val_loss', verbose=1, patience=5)
  model.fit([X1_train, X2_train], Y_train, 
            nb_epoch=50, batch_size=8
            , validation_split=0.1
            , callbacks=[early]
            , shuffle=True)
  score = model.evaluate([X1_val, X2_val], Y_val, batch_size=16)
  print(score)

  transforms = [
            # Build basic folds first
            # ht.Fold(\"Mul > Add > Sum\", \"MulAddSum\")
            # Display fully-connected layers differently
            ht.Prune("IsVariableInitialized"),
            # Fold repeated nodes
            ht.FoldDuplicates()
            ]
  K.set_learning_phase(1)
  hl_graph = hl.build_graph(g) #, transforms=transforms)
  hl_graph.theme = hl.graph.THEMES['blue'].copy()
  hl_graph.save('/tmp/filediag.pdf')  # Display graph view



self.W: <tf.Variable 'neural_tensor_diag_layer/W:0' shape=(24, 32) dtype=float32>
e1: Tensor("neural_tensor_diag_layer/Tile:0", shape=(None,), dtype=float32)
e1: Tensor("neural_tensor_diag_layer/Reshape_2:0", shape=(None, 32, 24), dtype=float32)
x0: Tensor("neural_tensor_diag_layer/Reshape_6:0", shape=(None, 32, 32), dtype=float32)


ValueError: ignored

In [0]:
print('val #:', X1_val.shape[0])
score = model.evaluate([X1_val, X2_val], Y_val, batch_size=16)

files.download('/tmp/filediag.pdf')
